### Import

In [1]:
import os
import time
import numpy as np
from tqdm import tqdm
from timm import create_model
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable
import torchvision
from torchvision.transforms import ToTensor, Compose, Resize, Normalize
import torchvision.models as models
import matplotlib.pyplot as plt

c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data



In [2]:
transform = Compose([
    Resize((224, 224)),  # 調整到模型需求的輸入大小
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 常見正規化數值
])

train_ds = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/train', transform=transform)
valid_ds = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/vali_20k', transform=transform)
test_ds  = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/test_50k', transform=transform)

### Model

In [3]:
NUM_CLASSES = len(train_ds.class_to_idx)

alexnet = models.alexnet(pretrained=False, num_classes=NUM_CLASSES)

vgg16 = models.vgg16(pretrained=False)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, NUM_CLASSES)
batch_norm = False  # 預設為 False
init_weights = True  # 預設為 True

####################### 需要額外處理 #######################
googlenet = models.googlenet(pretrained=False, aux_logits=True, transform_input=False)# 3. GoogleLeNet (Inception v1)
googlenet.fc = nn.Linear(googlenet.fc.in_features, NUM_CLASSES)
googlenet.aux1.fc2 = nn.Linear(googlenet.aux1.fc2.in_features, NUM_CLASSES)  # 修改輔助分類器
googlenet.aux2.fc2 = nn.Linear(googlenet.aux2.fc2.in_features, NUM_CLASSES)

resnet18 = models.resnet18(pretrained=False, zero_init_residual=False)
resnet18.fc = nn.Linear(resnet18.fc.in_features, NUM_CLASSES)

squeezenet = models.squeezenet1_0(pretrained=False, num_classes=NUM_CLASSES)
squeezenet.classifier[1] = nn.Conv2d(512, NUM_CLASSES, kernel_size=(1, 1), stride=(1, 1))

mobilenet_v2 = models.mobilenet_v2(pretrained=False, width_mult=1.0, round_nearest=8)
mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features, NUM_CLASSES)

efficientnet_b0 = models.efficientnet_b0(pretrained=False)
efficientnet_b0.classifier[1] = nn.Linear(efficientnet_b0.classifier[1].in_features, NUM_CLASSES)

swin_transformer = models.swin_t(pretrained=False, num_classes=NUM_CLASSES)
swin_transformer.head = nn.Linear(swin_transformer.head.in_features, NUM_CLASSES)
embed_dim = 96  # 預設
depths = [2, 2, 6, 2]  # 預設
num_heads = [3, 6, 12, 24]  # 預設
window_size = 7  # 預設
drop_path_rate = 0.1  # 預設
ape = False  # 預設
patch_norm = True  # 預設

xception = create_model('xception', pretrained=False, num_classes=NUM_CLASSES, drop_rate=0.2)

convnext_tiny = create_model('convnext_tiny', pretrained=False, num_classes=NUM_CLASSES, drop_path_rate=0.1)

mnasnet = models.mnasnet1_0(pretrained=False)
mnasnet.classifier[1] = nn.Linear(mnasnet.classifier[1].in_features, NUM_CLASSES)

c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\timm\models\_factory.py:117: UserWarning:

### Training Parameters

In [4]:
# Model
model = alexnet

EPOCHS = 1
BATCH_SIZE = 256
LEARNING_RATE = 1e-3   
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 
loss_func = nn.CrossEntropyLoss()

num_workers = 4

### Setup

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.cuda()

### Train

In [6]:
# 儲存訓練損失與準確度
train_losses = []
train_accuracies = []

# 訓練
start_time = time.time()
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    # 使用 tqdm 包裹 train_loader
    train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")
    for step, (x, y) in train_bar:
        x, y = x.to(device), y.to(device)

        # Forward
        output = model(x)
        loss = loss_func(output, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 累計損失與準確度
        train_loss += loss.item() * x.size(0)
        _, preds = torch.max(output, 1)
        correct_train += (preds == y).sum().item()
        total_train += y.size(0)

        # 更新進度條
        train_bar.set_postfix({"loss": loss.item()})

    # 計算單個 epoch 的平均損失與準確度
    epoch_loss = train_loss / len(train_ds)
    epoch_accuracy = correct_train / total_train

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch {epoch + 1}: Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

training_time = time.time() - start_time

# 繪製損失與準確度圖表
epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(12, 5))

# 損失圖
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train Loss")
plt.legend()

# 準確度圖
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label="Train Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train Accuracy")
plt.legend()

plt.tight_layout()
plt.show()


Number of train samples:  958941
Number of test samples:  49500
Detected Classes are:  {'dry_asphalt_severe': 0, 'dry_asphalt_slight': 1, 'dry_asphalt_smooth': 2, 'dry_concrete_severe': 3, 'dry_concrete_slight': 4, 'dry_concrete_smooth': 5, 'dry_gravel': 6, 'dry_mud': 7, 'fresh_snow': 8, 'ice': 9, 'melted_snow': 10, 'water_asphalt_severe': 11, 'water_asphalt_slight': 12, 'water_asphalt_smooth': 13, 'water_concrete_severe': 14, 'water_concrete_slight': 15, 'water_concrete_smooth': 16, 'water_gravel': 17, 'water_mud': 18, 'wet_asphalt_severe': 19, 'wet_asphalt_slight': 20, 'wet_asphalt_smooth': 21, 'wet_concrete_severe': 22, 'wet_concrete_slight': 23, 'wet_concrete_smooth': 24, 'wet_gravel': 25, 'wet_mud': 26}
Epoch 1/1


Training: 100%|██████████| 3746/3746 [30:29<00:00,  2.05it/s, loss=1.07] 


### Save Model


In [9]:
# 測試準確度
test_x, test_y = next(iter(test_loader))
test_x, test_y = test_x.to(device), test_y.to(device)
test_output = model(test_x).argmax(1)
accuracy = (test_output == test_y).sum().item() / BATCH_SIZE

# 模型名稱
model_name = type(model).__name__

# 檔案名稱包含訓練時間
file_name = f"{model_name}_ACC{accuracy:.2f}_E{EPOCHS}_BS{BATCH_SIZE}_LR{LEARNING_RATE:.0e}_T{int(training_time // 3600):02d}h{int((training_time % 3600) // 60):02d}m{int(training_time % 60):02d}s.pt"

# 儲存路徑
save_path = os.path.join('../2_results/0_weight', file_name)
torch.save(model, save_path)

print(f"Model saved as: {file_name}")

Model saved as: AlexNet_ACC0.54_E1_BS256_LR1e-03_T00h30m46s.pt
